In [2]:
# Instalação das bibliotecas necessárias
!pip install --upgrade crewai crewai-tools

# Imports
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from google.colab import userdata

In [4]:
# Configuração de API
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Ferramentas
search_tool = SerperDevTool()

# Definição dos Agentes

# 1. Agente de Busca de Produto
product_search_agent = Agent(
    role='Pesquisador de Varejo',
    goal='Encontrar cartuchos de tinta HP em sites de varejo específicos (Amazon, Mercado Livre) e coletar resultados de busca brutos.',
    backstory=(
        "Um especialista em encontrar produtos online usando consultas de busca avançadas (Google dorks) "
        "para focar em sites específicos e obter os dados mais relevantes."
    ),
    tools=[search_tool],
    verbose=True
)

# 2. Agente Extrator de Dados
data_extractor_agent = Agent(
    role='Extrator de Dados de Produto',
    goal='Analisar os resultados brutos da busca (JSON/texto) e extrair informações chave: Nome do Produto, Preço, Avaliação, URL e Disponibilidade.',
    backstory=(
        "Um analista meticuloso que transforma dados de busca não estruturados ou semi-estruturados "
        "em informações claras e formatadas, pronto para análise."
    ),
    tools=[],
    verbose=True
)

# 3. Agente Consolidador (CSV)
csv_consolidator_agent = Agent(
    role='Compilador de CSV',
    goal='Compilar os dados estruturados de múltiplos produtos em um único arquivo CSV, garantindo a formatação correta.',
    backstory=(
        "Um organizador de dados eficiente, focado em criar relatórios CSV padronizados e prontos para uso, "
        "seguindo estritamente o cabeçalho solicitado."
    ),
    tools=[],
    verbose=True
)



In [5]:
# Definição das Tarefas

# Tarefa 1: Buscar os produtos
search_task = Task(
    description=(
        "Realize buscas para os seguintes 3 cartuchos HP. Para cada cartucho, "
        "busque tanto na 'amazon.com.br' quanto no 'mercadolivre.com.br'.\n"
        "Produtos para buscar:\n"
        "1. 'Cartucho HP 664 Preto'\n"
        "2. 'Cartucho HP 667 Colorido'\n"
        "3. 'Cartucho HP 662 Preto'\n\n"
        "Exemplos de consultas a serem feitas: "
        "'Cartucho HP 664 Preto site:amazon.com.br', "
        "'Cartucho HP 664 Preto site:mercadolivre.com.br', "
        "'Cartucho HP 667 Colorido site:amazon.com.br', etc.\n"
        "Colete os 2-3 principais resultados para CADA consulta."
    ),
    expected_output=(
        "Um relatório de texto compilado com os resultados brutos (snippet, source, price, url) "
        "de todas as buscas realizadas. Separe claramente os resultados de cada produto."
    ),
    agent=product_search_agent
)

# Tarefa 2: Extrair os dados
extraction_task = Task(
    description=(
        "Analise o relatório de resultados da busca fornecido. Para CADA produto/link encontrado, "
        "extraia as seguintes informações:\n"
        "- Nome do Produto (seja o mais claro possível)\n"
        "- Preço (R$) (Formate como 'R$ XX,XX'. Se não encontrar, coloque 'N/D')\n"
        "- Avaliação (estrelas) (Formate como 'X.X estrelas'. Se não encontrar, coloque 'N/D')\n"
        "- URL (O link direto para o produto)\n"
        "- Disponibilidade (Ex: 'Em estoque', 'Indisponível'. Se não for claro, coloque 'Verificar no site')\n"
    ),
    expected_output=(
        "Uma lista estruturada (pode ser um JSON ou uma lista de strings formatadas) "
        "contendo os dados extraídos para todos os produtos encontrados. "
        "Ex: [{'Nome do Produto': '...', 'Preço (R$)': '...', ...}, ...]"
    ),
    agent=data_extractor_agent,
    context=[search_task]
)

# Tarefa 3: Consolidar em CSV
csv_creation_task = Task(
    description=(
        "Converta a lista estruturada de produtos em um formato CSV. "
        "O cabeçalho do CSV DEVE ser exatamente:\n"
        "Nome do Produto,Preço (R$),Avaliação (estrelas),URL,Disponibilidade\n\n"
        "Certifique-se de que cada produto esteja em uma nova linha e que os dados "
        "estejam corretamente alinhados com as colunas."
    ),
    expected_output="Um arquivo 'cartuchos_hp.csv' contendo os dados formatados.",
    agent=csv_consolidator_agent,
    context=[extraction_task],
    output_file='cartuchos_hp.csv'
)



In [6]:
# Criação e Execução da Crew
product_scraper_crew = Crew(
    agents=[product_search_agent, data_extractor_agent, csv_consolidator_agent],
    tasks=[search_task, extraction_task, csv_creation_task],
    process=Process.sequential,
    verbose=True
)

print("Iniciando a Crew de Coleta de Dados de Cartuchos HP")
result = product_scraper_crew.kickoff()

print("\n\n RESULTADO FINAL DA EXECUÇÃO")
print(result)

print("\n\nConteúdo do CSV Gerado (cartuchos_hp.csv)")
try:
    with open('cartuchos_hp.csv', 'r', encoding='utf-8') as f:
        print(f.read())
except FileNotFoundError:
    print("Arquivo 'cartuchos_hp.csv' não foi encontrado. A execução pode ter falhado.")

Iniciando a Crew de Coleta de Dados de Cartuchos HP


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6cc59ecf-5839-433b-95a0-3c05c744c814                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Pesquisador de Varejo                                                                                   │
│                                                                                                                 │
│  Task: Realize buscas para os seguintes 3 cartuchos HP. Para cada cartucho, busque tanto na 'amazon.com.br'     │
│  quanto no 'mercadolivre.com.br'.                                                                               │
│  Produtos para buscar:                                                                                          │
│  1. 'Cartucho HP 664 Preto'                                                                                     │
│  2. 'Cartucho HP 667 Colorido'                                                                                  │
│  3. 'Cartucho HP 662 Preto'                                                                                     │
│                                                                                                                 │
│  Exemplos de consultas a serem feitas: 'Cartucho HP 664 Preto site:amazon.com.br', 'Cartucho HP 664 Preto       │
│  site:mercadolivre.com.br', 'Cartucho HP 667 Colorido site:amazon.com.br', etc.                                 │
│  Colete os 2-3 principais resultados para CADA consulta.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Pesquisador de Varejo                                                                                   │
│                                                                                                                 │
│  Thought: Thought: I will perform searches for each of the three HP cartridge models on both Amazon and         │
│  Mercado Livre to gather relevant results.                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compilador de CSV                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  Nome do Produto,Preço (R$),Avaliação (estrelas),URL,Disponibilidade                                            │
│  Cartucho HP 664                                                                                                │
│  Preto,N/D,N/D,https://www.amazon.com.br/HP-F6V29AB-Cartucho-Tinta-Preto/dp/B074HMBWQG,Verificar no site        │
│  Cartucho HP 664 Preto Original (F6V31AB),N/D,4,7                                                               │
│  estrelas,https://www.amazon.com.br/cartucho-hp-664/s?k=cartucho+hp+664,Verificar no site                       │
│  Kit Cartuchos 664 Preto e colorido Original,N/D,4,7                                                            │
│  estrelas,https://www.amazon.com.br/Cartuchos-Preto-colorido-Original-1115/dp/B07Z23F33D,Verificar no site      │
│  Cartucho 664,R$187,99,4,8 estrelas,https://lista.mercadolivre.com.br/cartucho-664,Verificar no site            │
│  Cartucho HP 667 Colorido                                                                                       │
│  Original,N/D,N/D,https://www.amazon.com.br/Cartucho-HP-Colorido-Original-3YM79AB/dp/B08WKRYQHN,Verificar no    │
│  site                                                                                                           │
│  Kit 2 cartucho HP 667 Preto e Colorido,N/D,4,7                                                                 │
│  estrelas,https://www.amazon.com.br/Cartucho-Tinta-HP-3Ym78AL-Color/dp/B00I3FMUZ2,Verificar no site             │
│  Cartucho HP 662 Preto,N/D,N/D,https://www.amazon.com.br/cartucho-hp-662/s?k=cartucho+hp+662,Verificar no site  │
│  Kit Hp 662 Preto + Hp 662                                                                                      │
│  Colorido,N/D,N/D,https://produto.mercadolivre.com.br/MLB-2685231467-kit-hp-662-preto-hp-662-color-deskjet-151  │
│  6-2516-3516-4646-_JM,Verificar no site                                                                         │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0babc286-9be0-424b-a7bc-2481ac8086e4                                                                     │
│  Agent: Compilador de CSV                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6cc59ecf-5839-433b-95a0-3c05c744c814                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  Nome do Produto,Preço (R$),Avaliação (estrelas),URL,Disponibilidade                                            │
│  Cartucho HP 664                                                                                                │
│  Preto,N/D,N/D,https://www.amazon.com.br/HP-F6V29AB-Cartucho-Tinta-Preto/dp/B074HMBWQG,Verificar no site        │
│  Cartucho HP 664 Preto Original (F6V31AB),N/D,4,7                                                               │
│  estrelas,https://www.amazon.com.br/cartucho-hp-664/s?k=cartucho+hp+664,Verificar no site                       │
│  Kit Cartuchos 664 Preto e colorido Original,N/D,4,7                                                            │
│  estrelas,https://www.amazon.com.br/Cartuchos-Preto-colorido-Original-1115/dp/B07Z23F33D,Verificar no site      │
│  Cartucho 664,R$187,99,4,8 estrelas,https://lista.mercadolivre.com.br/cartucho-664,Verificar no site            │
│  Cartucho HP 667 Colorido                                                                                       │
│  Original,N/D,N/D,https://www.amazon.com.br/Cartucho-HP-Colorido-Original-3YM79AB/dp/B08WKRYQHN,Verificar no    │
│  site                                                                                                           │
│  Kit 2 cartucho HP 667 Preto e Colorido,N/D,4,7                                                                 │
│  estrelas,https://www.amazon.com.br/Cartucho-Tinta-HP-3Ym78AL-Color/dp/B00I3FMUZ2,Verificar no site             │
│  Cartucho HP 662 Preto,N/D,N/D,https://www.amazon.com.br/cartucho-hp-662/s?k=cartucho+hp+662,Verificar no site  │
│  Kit Hp 662 Preto + Hp 662                                                                                      │
│  Colorido,N/D,N/D,https://produto.mercadolivre.com.br/MLB-2685231467-kit-hp-662-preto-hp-662-color-deskjet-151  │
│  6-2516-3516-4646-_JM,Verificar no site                                                                         │
│  ```                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

 RESULTADO FINAL DA EXECUÇÃO
```
Nome do Produto,Preço (R$),Avaliação (estrelas),URL,Disponibilidade
Cartucho HP 664 Preto,N/D,N/D,https://www.amazon.com.br/HP-F6V29AB-Cartucho-Tinta-Preto/dp/B074HMBWQG,Verificar no site
Cartucho HP 664 Preto Original (F6V31AB),N/D,4,7 estrelas,https://www.amazon.com.br/cartucho-hp-664/s?k=cartucho+hp+664,Verificar no site
Kit Cartuchos 664 Preto e colorido Original,N/D,4,7 estrelas,https://www.amazon.com.br/Cartuchos-Preto-colorido-Original-1115/dp/B07Z23F33D,Verificar no site
Cartucho 664,R$187,99,4,8 estrelas,https://lista.mercadolivre.com.br/cartucho-664,Verificar no site
Cartucho HP 667 Colorido Original,N/D,N/D,https://www.amazon.com.br/Cartucho-HP-Colorido-Original-3YM79AB/dp/B08WKRYQHN,Verificar no site
Kit 2 cartucho HP 667 Preto e Colorido,N/D,4,7 estrelas,https://www.amazon.com.br/Cartucho-Tinta-HP-3Ym78AL-Color/dp/B00I3FMUZ2,Verificar no site
Cartucho HP 662 Preto,N/D,N/D,